On peut faire les fonctions impératives pour ce projet

In [2]:
import math
import random as rd
import sys
def algo_Euclide(a,b):
    if(b==0):
        return a
    else:
        return algo_Euclide(b,a%b)
        
def exponentiation_modulaire (a, e, n):
    p=1
    while e>0:
        if (e % 2 != 0):
            p=(p*a)%n
            a=(a*a)%n
            e=e//2
    return p


In [4]:
def algo_Euclide_etendu(a,b):
    if(b==0):
        return(a,1,0)
    else:
        d,u,v=algo_Euclide_etendu(b,a%b)
        return d,v,u-(a//b)*v
        

def gen_pochon(n):
    tab = []
    somme = 0
    for _ in range(n):
        b = rd.randint(somme + 1, somme + sys.maxsize // n)
        somme += b
        tab.append(b)
    return tab
def gen_W(M):
    W=rd.randint(1, M-1)
    while(math.gcd(W,M)!=1):
        W=rd.randint(1, M-1)
    return W
def gen_permutation(n):
    sigma = list(range(1, n+1))
    rd.shuffle(sigma)
    return sigma
    
def gen_A(pochon,W,M,sigma,n):
    tabA=[]
    i=0
    while (len(tabA)<n):
        tabA.append((pochon[sigma[i]]*W)%M)
        i+=1
    return tabA


n=5
pochon=[2,5,11,23,55]
M=113
W=27
sigma=[1,4,2,0,3]
A=gen_A(pochon,W,M,sigma,n)
def solve_pochon(pochon,c,i):
    if(i==0):
        return 0
    elif(pochon[i]<=c):
        return pochon[i]+solve_pochon(pochon,c-pochon[i], i-1)
    else:
        return solve_pochon(pochon,c, i-1)
